### Importación de bibliotecas y carga de datos

En esta sección se importan las bibliotecas necesarias (pandas, numpy, matplotlib.pyplot, bnlearn) y se carga el archivo de datos 'exoplanets.xls' utilizando pandas.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import bnlearn as bn

In [ ]:
df = pd.read_excel('exoplanets.xls')

### Categorización de variables reales

Aquí se define la función categorize_variable que toma un dataframe, una variable y una lista de etiquetas, y categoriza la variable en el dataframe utilizando el método qcut de pandas. Se realizan varias llamadas a esta función para categorizar diferentes variables del dataframe.

In [ ]:
def categorize_variable(dataframe: pd.DataFrame, variable: str, labels: list) -> None:
    dataframe[variable] = pd.qcut(x=dataframe[variable], q=3, labels=labels)
    return None

In [ ]:
categorize_variable(df, 'radius (m)', ["Small", "Medium", "Big"])
categorize_variable(df, 'mass (kg)', ["Light", "Medium", "Heavy"])
categorize_variable(df, 'gravity', ["Weak", "Medium", "Strong"])
categorize_variable(df, 'avg_temp (C)', ["Cold", "Tempered", "Warm"])
categorize_variable(df, 'rotational_period (h)', ["Slow", "Normal", "Fast"])

### Obtención y reflexión sobre el grafo de la red bayesiana

En esta sección se utiliza la librería bnlearn para obtener el grafo de manera automática mediante el método 'hc' (Hill Climbing) y el scoretype 'bdeu' (Bayesian Dirichlet Equivalent Uniform). Luego se imprime el grafo y se reflexiona sobre su verosimilitud.

In [ ]:
DAG = bn.structure_learning.fit(df, methodtype='hc', scoretype='bdeu')
bn.plot(DAG)

### Creación y visualización del grafo de la red bayesiana

Se definen las conexiones entre las variables del modelo mediante una lista de tuplas llamada model_edges. Cada tupla representa una conexión entre dos variables. Se explican las relaciones entre las variables en los comentarios.

Se utiliza la función make_DAG de bnlearn para crear un grafo dirigido acíclico (DAG) a partir de las conexiones definidas en model_edges. Luego se imprime el grafo.

In [ ]:
model_edges = [
    ('radius (m)', 'mass (kg)'), # La masa depende de la cantidad de materia en el objeto y su tamaño
    ('mass (kg)', 'gravity'), # La gravedad depende de la cantidad de materia en el objeto
    ('radius (m)', 'gravity'), # La gravedad también depende del tamaño del objeto
    ('mass (kg)', 'atmosphere'), # La masa puede afectar la capacidad del objeto para retener una atmósfera.
    ('gravity', 'atmosphere'), # La gravedad puede afectar la capacidad del objeto para retener una atmósfera.
    ('atmosphere', 'oxygene'), # La presencia de una atmósfera adecuada es un factor importante para la existencia de oxígeno respirable
    ('atmosphere', 'water'), # La presencia de una atmósfera adecuada es un factor importante para la existencia de agua líquida
    ('atmosphere', 'life'),  # La atmósfera es un factor esencial para la vida
    ('oxygene', 'life'), # El oxígeno es un factor esencial para la vida
    ('water', 'life') # El agua es un factor esencial para la vida
]

DAG_life = bn.make_DAG(model_edges)

bn.plot(DAG_life, interactive=False)

### Aprendizaje de los parámetros y obtención de las tablas de probabilidad condicionada

Se utiliza la función parameter_learning.fit de bnlearn para aprender los parámetros del modelo a partir de los datos del dataframe y se imprime la tabla de probabilidad condicionada resultante.

In [ ]:
model = bn.parameter_learning.fit(DAG, df)

In [ ]:
bn.print_CPD(model)

### Realización de inferencias sobre propiedades de los planetas

Se realizan varias inferencias utilizando la función inference.fit de bnlearn. Se especifican diferentes variables objetivo y evidencia para realizar las inferencias y se imprime el resultado de cada inferencia en forma de tabla de probabilidad condicionada. Cada inferencia se identifica con un título (q1, q2, q3, q4) y se explican las condiciones de evidencia utilizadas en los comentarios.

In [ ]:
q1 = bn.inference.fit(model, variables=['life'], evidence={'water': 'YES', 'atmosphere': 'YES'})
print(q1.df)

In [ ]:
q2 = bn.inference.fit(model, variables=['life'], evidence={'water': 'NO', 'atmosphere': 'NO', 'oxygene': 'NO'})
print(q2.df)

In [ ]:
q3 = bn.inference.fit(model, variables=['life'], evidence={'mass (kg)': 'Light', 'radius (m)': 'Small'})
print(q3.df)

In [ ]:
q4 = bn.inference.fit(model, variables=['life'], evidence={'mass (kg)': 'Heavy', 'radius (m)': 'Big'})
print(q4.df)